<a href="https://colab.research.google.com/github/AVVSEE/AVVSEE/blob/main/Background_Color_Picker_k_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Background color picker

**How to Use:**

Upload Image
& View Results:

    1.   Locate the cell with the description "Upload Image"
    
    2.   Choose your desired image from your computer.


**Once the image is uploaded, the notebook will automatically process it.**
The uploaded image will be displayed, followed by its dominant colors and complementary colors.
The colors are presented as rectangular blocks with labels indicating whether they're dominant or complementary.

**To analyze a different image**, simply use the "Upload" button again to select a new image. The results will update automatically.
Technical Details:

`Dominant Colors`: These are extracted using the **K-means clustering algorithm**, *which groups pixels in the image based on color*. The center of each cluster represents a dominant color.
Complementary Colors: For each dominant color, the complementary color is calculated by taking the inverse in the RGB color space.


*Notes:*
*The tool currently extracts three dominant colors from each image. This number can be adjusted in the code if needed.*

*The quality and accuracy of the results may vary depending on the image. For best results, use clear images with distinguishable colors*

**imports**



*   `matplotlib` for output display
*   `cv2` for img reading
*  ` numpy/sklearn` for k-means clustering









In [11]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from sklearn.cluster import KMeans

Added an easy interface for file upload using `ipywidgets`'s `IPython.display`

In [12]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# File upload widget
uploader = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description="Upload Image"
)

output = widgets.Output()

def on_upload_change(change):
    # Clear any previous output
    clear_output(wait=True)
    display(uploader, output)

    # Get uploaded file content
    uploaded_file = next(iter(uploader.value.values()))
    content = uploaded_file['content']

    # Convert bytes to numpy array and display the image
    nparr = np.frombuffer(content, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    with output:
        clear_output(wait=True)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img_rgb)
        plt.axis('off')
        plt.title("Uploaded Image")
        plt.show()

        # Process the image and display results
        dominant_colors = get_dominant_colors(img_rgb)
        complementary_colors = [complementary_color(color) for color in dominant_colors]
        plot_colors(dominant_colors, ["Dominant 1", "Dominant 2", "Dominant 3"])
        plot_colors(complementary_colors, ["Complementary 1", "Complementary 2", "Complementary 3"])

uploader.observe(on_upload_change, names='_counter')

display(uploader, output)

FileUpload(value={'nerv3wpp.png': {'metadata': {'name': 'nerv3wpp.png', 'type': 'image/png', 'size': 242111, '…

Output()

**Read image with openCV**
**Manage oversized files**

In [16]:
def load_and_display_image(image_path, max_size=500):
    """Load the image using OpenCV, resize if necessary, and display it."""
    img = cv2.imread(image_path)
    # Resize the image if its largest dimension is greater than max_size
    h, w = img.shape[:2]
    if max(h, w) > max_size:
        if h > w:
            new_h, new_w = max_size, int(max_size * w / h)
        else:
            new_h, new_w = int(max_size * h / w), max_size
        img = cv2.resize(img, (new_w, new_h))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()
    return img_rgb

**Extract dominant colors using K-means clustering**

In [14]:
def get_dominant_colors(img, k=3):
    """Extract k dominant colors from the image using K-means clustering."""
    pixels = img.reshape(-1, 3)
    if pixels.shape[1] == 4:
        pixels = pixels[:, :3]
    kmeans = KMeans(n_clusters=k,n_init=10)
    kmeans.fit(pixels)
    return kmeans.cluster_centers_

def complementary_color(rgb):
    """Find the complementary color for a given RGB value."""
    r, g, b = rgb
    return (255 - r, 255 - g, 255 - b)

**plotting dominant & complementary colors**

In [15]:
def plot_colors(colors, titles):
    """Plot a series of colors in a row."""
    fig, axs = plt.subplots(1, len(colors), figsize=(len(colors)*2, 2),
                            subplot_kw=dict(xticks=[], yticks=[], frame_on=False))
    for ax, color, title in zip(axs, colors, titles):
        normalized_color = [c/255 for c in color]
        ax.imshow([[normalized_color]], aspect='auto')
        ax.set_title(title, fontdict={'fontsize': 12})
    plt.tight_layout()
    plt.show()

def process_image(image_path):
    """Load the image, extract dominant colors, compute complementary colors, and visualize the results."""
    img_rgb = load_and_display_image(image_path)
    dominant_colors = get_dominant_colors(img_rgb)
    complementary_colors = [complementary_color(color) for color in dominant_colors]

    plot_colors(dominant_colors, ["Dominant 1", "Dominant 2", "Dominant 3"])
    plot_colors(complementary_colors, ["Complementary 1", "Complementary 2", "Complementary 3"])

**Error workaround** : To suppress the warning and ensure consistent behavior, you can explicitly set n_init=10 (*or any other value -useless to go above 100 in this use case*) when calling KMeans.

n_init currently set to **10**

*made by leo muller 10oct2023*